# space

In [1]:

import torch
import logging

import nni.retiarii.nn.pytorch as nn

from collections import OrderedDict
from nni.retiarii import model_wrapper


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Convolutions(nn.Module):
    def __init__(self, conv, layer_name):
        super().__init__()

        self.conv = nn.LayerChoice(conv, label=f'{layer_name} - Step 2: Convolutions, Batchnorm and Activation')

    def forward(self, x):
        x = self.conv(x)
        return x
    
class BaseBlock(nn.Module):
    def __init__(self):
        super(BaseBlock, self).__init__()

    def get_conv_ordered_dict(self, in_channels, out_channels, ks, pd, dl, activation):
        layers = [
            ("Conv2d", nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl),
                nn.BatchNorm2d(out_channels),
                activation,
                nn.Conv2d(out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl),
                nn.BatchNorm2d(out_channels),
                activation
                )
            ),
            ("DepthwiseSeparable", nn.Sequential(
                nn.Conv2d(in_channels, in_channels, kernel_size=ks, padding=pd, dilation=dl, groups=in_channels),
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels),
                activation,
                nn.Conv2d(out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl, groups=out_channels),
                nn.Conv2d(out_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels),
                activation
                )
            )
        ]
        return OrderedDict(layers)

    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]

class EncoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, downsamples, layer_name):
        super(EncoderBlock, self).__init__()
        
        self.downsample = nn.LayerChoice(downsamples,label=f'{layer_name} - Step 1: Downsampling Technique')
        self.conv_layer = Convolutions(self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl, activations), layer_name)

    def forward(self, x):
        x = self.downsample(x)
        x = self.conv_layer(x)
        return x

class DecoderBlock(BaseBlock):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, upsamples, layer_name):
        super(DecoderBlock, self).__init__()

        self.upsample = nn.LayerChoice(upsamples, label=f"{layer_name} - Step 0: Upsampling Technique")
        self.conv_layer = Convolutions(self.get_conv_ordered_dict(in_channels, out_channels, ks, pd, dl, activations), layer_name)

    def forward(self, x, skip):
        upsampled = self.upsample(x)
        cropped = self.crop_tensor(upsampled, skip)
        return self.conv_layer(torch.cat([cropped, upsampled], 1))

@model_wrapper
class SearchSpace(BaseBlock):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()

        network_depth = nn.ValueChoice([1, 2, 3, 4], label="Network Depth")

        ks = nn.ValueChoice([3, 5], label="Kernel Size")
        dl = nn.ValueChoice([1, 3], label="Dilation Rate")
        pd = (ks - 1) * dl // 2

        activations = nn.LayerChoice(OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            # ("Sigmoid", nn.Sigmoid()),
            ("SiLU", nn.SiLU(inplace=True)),
        ]), label="Activation")

        downsamples = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsamples = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True))
        ])

        # Conv layer in"
        self.mid_channels = 64
        self.first = Convolutions(self.get_conv_ordered_dict(in_channels, self.mid_channels, 1, 0, 1, nn.Sigmoid()), "First Conv Layer")

        # For Encoders:
        encoder_block = lambda index: EncoderBlock(64*(2**index), 64*(2**(index+1)), ks, pd, dl, activations, downsamples, f"Encoder {index+1}")
        self.encoders = nn.Repeat(encoder_block, network_depth)

        # For Decoders:
        decoder_block = lambda index: DecoderBlock(64*(2**(index))*3, 64*(2**index), ks, pd, dl, activations, upsamples, f"Decoder {index+1}")
        self.decoders = nn.Repeat(decoder_block, network_depth)
        self.decoders = self.decoders[::-1]

        # Conv layer out
        self.out = nn.Conv2d(self.mid_channels, out_channels, kernel_size=1, padding=0, dilation=1)
        
    def forward(self, x):
        logger.info("Input: %s", x.size())
        
        # Variables to store intermediate values
        encoder_outputs = []

        # Start with the first conv layer
        x = self.first(x)
        encoder_outputs.append(x)
        logger.info(f"Initial Conv Layer: %s", x.size())

        # Encoder pass
        for i, encoder in enumerate(self.encoders):
            x = encoder(x)
            encoder_outputs.append(x)
            logger.info(f"Encoder {i+1}: %s", x.size())

        # Decoder pass
        for i, decoder in enumerate(self.decoders):
            x = decoder(x, encoder_outputs[-(i+2)])
            logger.info(f"Decoder {len(self.decoders) - i}: %s", x.size())

        x = self.out(x)
        logger.info("Output: %s", x.size())
        return x

# test1

In [ ]:

import nni.retiarii.strategy as strategy

from darts.eval import main_evaluation

from nni.experiment import Experiment
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig


In [ ]:
strategies = [
    strategy.Random(dedup=True), # multi
    strategy.RegularizedEvolution(dedup=True), # multi
    strategy.TPE(), # multi
    strategy.DARTS() # One-Shot
]

In [37]:
# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
# search_strategy = strategy.Random(dedup=True)
search_strategy = strategy.DARTS()
# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

# exp_config.training_service.engine = 'oneshot' # only if using darts


exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

[2023-08-16 16:28:31] Creating experiment, Experiment ID: dk71i4ha


2023-08-16 16:28:31,380 - INFO - Creating experiment, Experiment ID: ${CYAN}dk71i4ha


[2023-08-16 16:28:31] Starting web server...


2023-08-16 16:28:31,399 - INFO - Starting web server...


[2023-08-16 16:28:32] Setting up...


...

[2023-08-16 16:28:32] Start strategy...


2023-08-16 16:28:32,621 - INFO - Start strategy...


[2023-08-16 16:28:32] Successfully update searchSpace.


2023-08-16 16:28:32,664 - INFO - Successfully update searchSpace.


ValueError: Mutator is not empty. The reason might be that you have used the wrong execution engine. Try to set engine to `oneshot` and try again.

In [17]:
experiment = Experiment.connect(8081)
experiment.stop()

[2023-08-16 17:22:04] Connect to port 8081 success, experiment id is a0o93vrm, status is RUNNING.


2023-08-16 17:22:04,700 - INFO - Connect to port 8081 success, experiment id is a0o93vrm, status is RUNNING.


[2023-08-16 17:22:04] Stopping experiment, please wait...


2023-08-16 17:22:04,704 - INFO - Stopping experiment, please wait...


[2023-08-16 17:22:04] Experiment stopped


2023-08-16 17:22:04,713 - INFO - Experiment stopped


[2023-08-16 17:22:04] Dispatcher exiting...


2023-08-16 17:22:04,719 - INFO - Dispatcher exiting...


[2023-08-16 17:22:05] Dispatcher terminiated


2023-08-16 17:22:05,255 - INFO - Dispatcher terminiated


# test 2

In [5]:
from nni.nas.strategy import DARTS as DartsStrategy
# from nni.nas.experiment import RetiariiExperiment
from nni.nas.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.nas.evaluator import FunctionalEvaluator
from darts.eval import main_evaluation
from nni.experiment import Experiment


# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)


In [16]:

# search strategy
# search_strategy = strategy.Random(dedup=True)
search_strategy = DartsStrategy()

# experiment
# exp = RetiariiExperiment(model_space, evaluator, None, search_strategy)
exp = RetiariiExperiment(base_model=model_space, evaluator=evaluator, strategy=search_strategy)

exp_config = RetiariiExeConfig('local')
# exp_config.experiment_name = 'mnist_search'
# exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
# exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

# # exp_config.training_service.engine = 'oneshot' # only if using darts


# exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

exported_arch = experiment.export_top_models(formatter='dict')[0:10]

exported_arch[0]

[2023-08-16 17:08:59] Creating experiment, Experiment ID: a0o93vrm


2023-08-16 17:08:59,446 - INFO - Creating experiment, Experiment ID: ${CYAN}a0o93vrm


[2023-08-16 17:08:59] Starting web server...


2023-08-16 17:08:59,462 - INFO - Starting web server...


[2023-08-16 17:09:00] Setting up...


...

[2023-08-16 17:09:00] Start strategy...


2023-08-16 17:09:00,220 - INFO - Start strategy...


[2023-08-16 17:09:00] Successfully update searchSpace.


2023-08-16 17:09:00,269 - INFO - Successfully update searchSpace.


ValueError: Mutator is not empty. The reason might be that you have used the wrong execution engine. Try to set engine to `oneshot` and try again.

In [14]:
experiment = Experiment.connect(8081)
experiment.stop()

[2023-08-16 17:08:40] Connect to port 8081 success, experiment id is knb71v3r, status is RUNNING.


2023-08-16 17:08:40,038 - INFO - Connect to port 8081 success, experiment id is knb71v3r, status is RUNNING.


[2023-08-16 17:08:40] Stopping experiment, please wait...


2023-08-16 17:08:40,041 - INFO - Stopping experiment, please wait...


[2023-08-16 17:08:40] Experiment stopped


2023-08-16 17:08:40,059 - INFO - Experiment stopped


[2023-08-16 17:08:40] Dispatcher exiting...


2023-08-16 17:08:40,063 - INFO - Dispatcher exiting...


[2023-08-16 17:08:41] Dispatcher terminiated


2023-08-16 17:08:41,657 - INFO - Dispatcher terminiated
